In [1]:
import torch, gc
import time
import math

In [ ]:
#pytorch CPU

n = 8000
m1 = torch.randn(n,n)
m2 = torch.randn(n,n)
r = torch.randn(n,n)

start_time = time.time()
r = torch.matmul(m1,m2)

t1 = time.time() - start_time
print("pytorch CPU time "+str(n)+"x"+str(n),t1)

n = int(n/2)
m1 = torch.randn(n,n)
m2 = torch.randn(n,n)
r = torch.randn(n,n)

start_time = time.time()
r = torch.matmul(m1,m2)

t2 = time.time() - start_time
print("pytorch CPU time "+str(n)+"x"+str(n),t2)

x = t1 / t2
print(x,"times slower")

p = math.log(x) / math.log(2)

print("pytorch CPU -> n^" + str(p) + "\n\n")

#pytorch GPU

device = torch.device("cuda")

n = 24000
m1 = torch.randn(n,n).to(device)
m2 = torch.randn(n,n).to(device)
r = torch.randn(n,n).to(device)

start_time = time.time()
r = torch.matmul(m1,m2)

torch.cuda.synchronize()
t1 = time.time() - start_time
print("pytorch GPU time "+str(n)+"x"+str(n),t1)

n = int(n/2)

gc.collect()
torch.cuda.empty_cache()

m1 = torch.randn(n,n).to(device)
m2 = torch.randn(n,n).to(device)
r = torch.randn(n,n).to(device)

start_time = time.time()
r = torch.matmul(m1,m2)

torch.cuda.synchronize()
t2 = time.time() - start_time
print("pytorch GPU time "+str(n)+"x"+str(n),t2)

x = t1 / t2
print(x,"times slower")

p = math.log(x) / math.log(2)

print("pytorch GPU -> n^" + str(p))
